<a href="https://colab.research.google.com/github/Agoplus/Advanced_Statistics-R/blob/main/settebello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Libraries
import numpy as np
import random as rd

In [6]:
# Game Definition
num_players = 6
deck_cards = 40
seed_1 = "coppe"
seed_2 = "bastoni"
seed_3 = "spade"
seed_4 = "denari"
jolly = {"value":10, "seed":seed_4}

# Strategy Parameters
max_player_call_value = np.arange(0,7.5,0.5)
max_dealer_call_value = np.arange(0,7.5,0.5)
max_player_jolly_call_value = np.arange(1,7,1)
max_dealer_jolly_call_value = max_player_jolly_call_value
strategy_parameters = {"max_player_call_value":max_player_call_value,
                   "max_dealer_call_value":max_dealer_call_value,
                   "max_player_jolly_call_value":max_player_jolly_call_value,
                   "max_dealer_jolly_call_value":max_dealer_jolly_call_value}

# **Functions**

In [3]:
def deck_shuffle():
  # Create a new deck with 4 seeds
  deck = {seed_1:list(i for i in range(1,11)), seed_2:list(i for i in range(1,11)), seed_3:list(i for i in range(1,11)), seed_4:list(i for i in range(1,11))}
  seeds = [seed_1, seed_2, seed_3, seed_4]
  return deck, seeds

In [4]:
def card_extraction(seeds, deck):
  # Check if there is still at least 1 card per seed in the deck
  for seed in seeds:
    if len(deck[seed]) == 0:
      del deck[seed] # If not -> removes the seed from the possible choices for the extraction
      seeds.remove(seed)
  # Choose a random face value and seed of the card
  card_seed = rd.choice(seeds)
  card_value = rd.choice(deck[card_seed])
  deck[card_seed].remove(card_value) # Removes the value/seed combination from the deck (the card is extracted)
  return Card(card_value, card_seed)

In [5]:
def settebello_value(value, seed):
  # Function to calculate the value of a card in "settebello" terms. If the card is the jolly then is counted as 7.5 for starting hand pourposes
  if value == jolly["value"] and seed == jolly["seed"]:
    return 7.5
  else:
    return value if value<8 else 0.5

In [6]:
def game(player_starting_hand_vector, dealer_starting_hand_vector, player_wins, dealer_wins, strategy_parameters, jolly):
  # Starts every game by shuffling the deck
  deck, seeds = deck_shuffle()
  players = []
  # Play the game by creating Player objects and Dealer object and giving them a starting Card object
  for player_id in range(0, num_players):
    player_starting_hand = card_extraction(seeds, deck)
    player_starting_hand_vector.append(settebello_value(player_starting_hand.value, player_starting_hand.seed))
    players.append(Player([player_starting_hand], player_id, seeds, deck, strategy_parameters, jolly))
  dealer_starting_hand = card_extraction(seeds, deck)
  dealer_starting_hand_vector.append(settebello_value(dealer_starting_hand.value, dealer_starting_hand.seed))
  dealer = Dealer([dealer_starting_hand], seeds, deck, strategy_parameters, jolly)
  # Player and Dealer strategy. Here is defined the max hand value until player and dealer call for an extra card.
  # If they arer below the threshold they draw using their method
  for player in players:
    while player.hand_value() < strategy_parameters["max_player_call_value"][10]:
      player.draw()
  while dealer.hand_value() < strategy_parameters["max_dealer_call_value"][11]:
    dealer.draw()
# Evaluation of winners. Results for the dealer are given ranging from 0 to 1 based on the fraction of player that they managed to beat
# Starting hand, corresponding result for players (binary 0 or 1) and dealer result (discrete value between 0 and 1) are saved in lists and dictionaries
  dealer_fraction_win = 0
  for player in players:
    if player.hand_value()<8:
      if dealer.hand_value()>7.5:
        player_wins.append(1)
      elif player.hand_value()>dealer.hand_value() and dealer.hand_value()<8:
        player_wins.append(1)
      else:
        player_wins.append(0)
        dealer_fraction_win = dealer_fraction_win + 1/num_players
    else:
      player_wins.append(0)
      dealer_fraction_win = dealer_fraction_win + 1/num_players
  dealer_wins.append(dealer_fraction_win)
  # Results are sent back to be given as input the game after to be updated with the next game
  return player_starting_hand_vector, dealer_starting_hand_vector, player_wins, dealer_wins

# **Classes**

In [7]:
class Card():
  def __init__(self, value, seed):
    self.value = value
    self.seed = seed

In [8]:
class Dealer():
  def __init__(self, hand, seeds, deck, strategy_parameters , jolly, re_denari = False):
    self.hand = hand
    self.seeds = seeds
    self.deck = deck
    self.jolly = jolly
    self.re_denari = re_denari
    self.max_player_jolly_call_value = strategy_parameters["max_player_jolly_call_value"][0]

  def hand_value(self):
    # Hand value is evaluated in 2 ways depending on if the jolly is present or not
    for card in self.hand:
      if [card.value, card.seed]==[self.jolly["value"],self.jolly["seed"]]:
        if self.re_denari==False:
          # If the jolly is present the hand is rearrenged and the jolly card is placed as first in the hand list
          self.re_denari = True
          self.hand.remove(card)
          self.hand.insert(0, card)
          break
    if self.re_denari:
      # If the jolly is present the hand value is given by considering the other cards in hand.
      # If the cards different from the jolly have a value below the max jolly call threshold the total value of the hand is considered 0.5 to
      # always allow an extra card to be called. The max jolly call threshold is part of the player/dealer strategy.
      # The value rule for the jolly is that it can assume any value of the other cards present in the deck (therefore not something like 4.5)
      value2 = 0
      for card2 in self.hand[1:]:
        value2 = value2 + settebello_value(card2.value, card2.seed)
      if value2%1==0 and value2<self.max_player_jolly_call_value+1:
        value = 0.5
      elif value2%1==0 and value2>self.max_player_jolly_call_value and value2<7:
        value = 7
      elif value2==7:
        value = 7.5
      elif value2%1!=0 and value2<8:
        value = 7.5
      else:
        value = 8
    else:
      # If the hand does not contain a jolly than the value is calculated applying normal settebello rules
      value = 0
      for card in self.hand:
        value = value + settebello_value(card.value, card.value)
    # If the value of the hand is above 7.5 it is considered a "bust" and a value of 8 is assigned. The player immidiatly loses the bet
    return value if value<8 else 8

  def draw(self):
    self.hand.append(card_extraction(self.seeds, self.deck))

In [9]:
class Player(Dealer):
  def __init__(self, hand, id, seeds, deck, strategy_parameters, jolly, re_denari = False):
    self.hand = hand
    self.id = id
    self.seeds = seeds
    self.deck = deck
    self.jolly = jolly
    self.re_denari = re_denari
    self.max_player_jolly_call_value = strategy_parameters["max_player_jolly_call_value"][0]

# **Game**

In [10]:
player_starting_hand_vector, dealer_starting_hand_vector, player_wins, dealer_wins = [],[],[],[]
i = 0
num_games = 100000
# Play games of the number of players chosen + dealer and saves results for both of them in lists and dictionaries
while i < num_games:
  player_starting_hand_vector, dealer_starting_hand_vector, player_wins, dealer_wins = game(player_starting_hand_vector, dealer_starting_hand_vector, player_wins, dealer_wins, strategy_parameters, jolly)
  i = i+1

# **Result Analysis**

In [11]:
starting_hand_result = {"0.5":[], "1":[], "2":[], "3":[], "4":[], "5":[], "6":[], "7":[], "7.5":[]}
average_winrate_starting_hand = {"0.5":[], "1":[], "2":[], "3":[], "4":[], "5":[], "6":[], "7":[], "7.5":[]}
# Aggregating results by starting hand of the players
for j in range(0,num_games):
  starting_card = player_starting_hand_vector[j]
  result = player_wins[j]
  starting_hand_result[str(starting_card)].append(result)

# Calculating average winrate based on starting hand for the players
for key in starting_hand_result:
  dict_sum = sum(starting_hand_result[key])
  dict_length = len(starting_hand_result[key])
  average_winrate_starting_hand[key].append(dict_sum/dict_length)

# Calculating average winrate of the dealer
sum = 0
for n in dealer_wins:
  sum = sum + n
print("Average dealer winrate:",sum/num_games)

for key in average_winrate_starting_hand:
  print("Average winrate for starting hand having value equal to",key,":",average_winrate_starting_hand[key][0])

Average dealer winrate: 0.6161550000000706
Average winrate for starting hand having value equal to 0.5 : 0.4148050328307839
Average winrate for starting hand having value equal to 1 : 0.334390800039655
Average winrate for starting hand having value equal to 2 : 0.29296836259049885
Average winrate for starting hand having value equal to 3 : 0.2463206046141607
Average winrate for starting hand having value equal to 4 : 0.23110979929161748
Average winrate for starting hand having value equal to 5 : 0.22641129032258064
Average winrate for starting hand having value equal to 6 : 0.5140930138916852
Average winrate for starting hand having value equal to 7 : 0.6626530202689143
Average winrate for starting hand having value equal to 7.5 : 0.7844998022933966


In [12]:
# Game
#deck, seeds = deck_shuffle()
#players = []
#for player_id in range(1, num_players+1):
  #players.append(Player([card_extraction(seeds, deck)], player_id, seeds, deck))
#dealer = Dealer([card_extraction(seeds, deck)], seeds, deck)

#for player in players:
  #print("Player",player.id,"starting hand:",player.hand_value())
  #while player.hand_value() < 5:
    #print(player.hand_value())
    #player.draw()
  #print("Player",player.id,"final hand:",player.hand_value())

#print("Dealer starting hand: ", dealer.hand_value())
#while dealer.hand_value() < 5:
  #print(dealer.hand_value())
  #dealer.draw()
#print("Dealer final hand: ",dealer.hand_value())

In [13]:
#winners = []
#for player in players:
  #if player.hand_value()<8:
    #if dealer.hand_value()>7.5:
      #winners.append(player.id)
    #elif player.hand_value()>dealer.hand_value() and dealer.hand_value()<8:
      #winners.append(player.id)
#print(winners)